# 神经信号的采集与人机智能技术概览课程作业——脑机接口数据分析
<center>韩诗雯 范可茗 翁逸杰</center>

## 一、数据预处理


### 1. 原始数据的缺陷

&emsp; 原始数据中，每个受试者共看了40张图片，在观察每张图片时，我们得到了19个通道在300Hz采样频率下获得的2100个数据点(共7s，每秒300个采样点)。直接进行分类会碰到曲线分类的通病——维度过高（此处有2100 samples $\times$ 19 channels $\times$ 40/10 trials 维度），因此要对其进行降维。同时，原始数据中的噪声很多，这就需要我们进行一定的处理，以获得原始数据中有价值的信息。



### 2. 对原始数据的处理

&emsp; 由于原始数据的维度高、噪声多，我们选择对其进行降维、滤波处理，具体如下：

#### 1）降维处理
&emsp; 首先，我们对原始信号进行截取，由于第三秒放出图片，且人类的反应时间一般在0.3s左右，因此我们取2.8s到3.8s。这也既保证了数据的完整性，也使得数据有了相当程度的降维（从2100到300）。

&emsp; 接着，我们对截取的数据进行特征提取，即计算四个频段的相对频段能量，把能量作为输入特征，接着再利用PCA对19个通道进行降维，我们截取了前五个主成分，把19个通道降为5个。但其实对于这个数据集，我们尝试过300维曲线直接分类，在扩增（见下）之后的稳定性以及精确度已经不错了，所以我们并未使用降维降到能量(从300到4)的数据集。

&emsp; 通过分析计算得出的能量以及PCA提取的第一主成分可以看出，不同人之间的看到图片后的大脑反映频段有所差异（代码见下）。在只有四个人的情况下，没有其余先天知识，我们很难模拟出一个适用性广泛的分类模型，所以在此只对一个人（以Person1为例）进行分类尝试。

#### 2）降噪以及频段截取

&emsp; 我们首先去除了比较明显的误差，如交流电、其他生物活动造成的生物电等。

&emsp; 频段截取同时有降噪以及特征提取的功能。我们尝试了DWT的Daubechies（示意图如下），以及傅立叶变换（fft）滤出beta，theta等波段。对于波段上的特征选择，我们可以使用DWT中的多种滤波方式得到很多波段，再利用PCA找到最佳的主成分波段；也可以使用fft直接滤出常见的波段，然后利用PCA对通道进行主成分分析，找到能量占比最高的波段。根据代码输出结果，我们可以看到4-13Hz波段能量占比最大，这与查找的文献结果相符合，所以我们截取了在这其中的theta波作为最终的输入特征。


#### 3）扩增数据集
&emsp; 在使用降维与降噪之后的数据集作为输入时，我们遇到了训练结果不稳定的问题。因为原来的数据集只有40个样本，还需要分割数据集，导致训练样本只有30个。但是截取后的曲线有300个点，在这样的高维空间之中30个样本过于分散，使用任何算法得到的决策边界都会距离测试样本非常远，所以算法的置信度很高（利用sklearn的predict_proba计算得出），但是精确度很随机。

&emsp; 因此，我们选择扩增数据集，即从2.7s到2.9s起，取61份1s内的数据，就是说对原有数据集进行时域上的平移。这样不仅简单地增加了样本的数量，还降低了人们在不同实验trial之中反应速度不同带来的误差。这样我们的训练样本从40扩展到了2440，并且实验得到的算法置信度有所降低，但是换来了非常稳定的精确度。

&emsp; 之后我们还计算了PCA降维之后的算法稳定性，同样降低了一点点算法的置信度，不过2240个数据集的KNN算法精确度已经比较高了（某些通道在0.9+），所以降维之后的精确度没有很大变化，我们还是使用扩增之后的300维曲线数据进行训练。

&emsp; 下面是我们相应的代码：

In [36]:
from scipy.io import loadmat
from scipy import signal

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import matplotlib.cm as cm
import pylab
import math
from matplotlib.collections import LineCollection
from matplotlib.ticker import MultipleLocator
from matplotlib.colors import ListedColormap
from scipy.fftpack import rfft, irfft, fftfreq, fft, ifft, fftshift

from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn import preprocessing
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.dummy import DummyClassifier
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from prettytable import PrettyTable

import torch

import ipympl

import pandas as pd

import pywt
import pywt.data

%matplotlib widget

## Channel Name

通道的名称

一共24个通道，rawTracePersonX之中只包含19个通道[1:8 10:16 19:20 23:24]，不包含9CM 17X3 18X2 21X1 22A2。cm（废弃通道），X1 X2 X3 A2为空通道（什么都没接）。

<img src="./source/pic1.png" width = "40%" />

In [37]:
ChanName = loadmat('./input/ChanName.mat')
chanName = np.array(ChanName['ChanName'])
chanNamex = chanName.reshape(24,)

chanName = []
for i in range(24):
    chanName.append(chanNamex[i][0][0])
chanName = np.array(chanName)

In [38]:
chanNameUsed = np.append(chanName[0:8], chanName[9:16])
chanNameUsed = np.append(chanNameUsed, chanName[18:20])
chanNameUsed = np.append(chanNameUsed, chanName[22:24])

chanNameUsed[11] = 'EEG T7 - Pz'  ## T3 is now T7
chanNameUsed[12] = 'EEG P7 - Pz'  ## T5 is now P7
chanNameUsed[-2] = 'EEG P8 - Pz'  ## T6 is now P8
chanNameUsed[-1] = 'EEG T8 - Pz'  ## T4 is now T8
chanNameUsed[10]
# 8 9 18 19 20 (21) 22 23 24 30 
# 35 36 37 40 47 52 53

'EEG Fp2 - Pz'

In [39]:
# timeRawTrace = loadmat('./input/timeRawTrace.mat')
# timeRawTrace = np.array(timeRawTrace['timeRawTrace'])
# timeRawTrace = timeRawTrace.reshape(2100,)

In [40]:
OSPerson1 = loadmat('./input/Person1/OSPerson1.mat')

# os1 = OSPerson1['OS'] # OS：要分析的数据，36x52x40x54 time X freq X Trial x pair
# time1 = OSPerson1['Time'] # why only 36?
# freq1 = OSPerson1['fOS'] # fOS：对应的频率
track1 = OSPerson1['Track'] # true label

In [41]:
# 通道：一共24个通道，此处只包含19个通道: [1:8 10:16 19:20 23:24] --> [1:8 9:15  16:17 18:19]
pair54 = np.array(loadmat('./input/Pair54.mat')['Pair54'])
for i in range(54):
    for j in range(2):
        if 10 <= pair54[i][j] <= 16:# & pair54[i][j] <= 16:
            pair54[i][j] -= 2
        elif 19 <= pair54[i][j] <= 20:
            pair54[i][j] -= 4
        elif 23 <= pair54[i][j] <= 24:
            pair54[i][j] -= 6
        else:
            pair54[i][j] -= 1
#     pass
pair54[19]

array([ 1, 18], dtype=uint8)

In [42]:
# rawTracePersonX
# dataTrail: 2100x40x19 时间 x Trial x 通道，包含了各通道各 Trail 的 Rawtrace
# track: 40个trial对应的图片编号。<11的编号为记忆过的图片，>10的是没有记忆过的。顺序和 trail 对应
rawTracePerson1 = loadmat('./input/Person1/rawTracePerson1.mat')
track1 = np.array(rawTracePerson1['Track']).reshape(40,)
dataTrial1 = np.array(rawTracePerson1['dataTrial'])

rawTracePerson2 = loadmat('./input/Person2/rawTracePerson2.mat')
track2 = np.array(rawTracePerson2['Track']).reshape(10,)
dataTrial2 = np.array(rawTracePerson2['dataTrial'])

rawTracePerson3 = loadmat('./input/Person3/rawTracePerson3.mat')
track3 = np.array(rawTracePerson3['Track']).reshape(40,)
dataTrial3 = np.array(rawTracePerson3['dataTrial'])

rawTracePerson4 = loadmat('./input/Person4/rawTracePerson4.mat')
track4 = np.array(rawTracePerson4['Track']).reshape(10,)
dataTrial4 = np.array(rawTracePerson4['dataTrial'])

## Preprocessing

&emsp; 首先，根据事实，我们将每次实验分为看到过图片和没有看到过两类。

&emsp; 其次，提取2.8-3.8s的数据信息。

&emsp; 最后，对原有数据进行时域上的平移(理由见上)。


In [75]:
# person 1 or 3

# 数据结构：
# data [data, pic, chanel] [300 x 2440 x 19]
# pair [pic, pair, data] [2440 x 54 x 300]

label_orgin = np.zeros([40])
for i in range(40):
    if track1[i] < 11:
        label_orgin[i] = 0
    if track1[i] > 10:
        label_orgin[i] = 1

label = label_orgin
data = dataTrial1[840:1140,:,:] # 提取序列2.8-3.8秒

# 平移扩增
for i in range(-30,30):
    label = np.append(label,label_orgin)
    data = np.concatenate( (data, dataTrial1[840+i:1140+i,:,:]), axis=1)


### PCA reduced data set

&emsp; 首先我们尝试使用DWT对数据进行滤波处理。

&emsp; 接下来，我们尝试滤波后计算各个频段的能量(power)占比，来对曲线进行特征提取。

&emsp; 这部分还展示了原始波形的时域图。



In [51]:
# 对于数据data[:,1,2], 尝试使用 DWT, Daubechies

t = np.linspace(0,1,300)
(cA, cD) = pywt.dwt(data[:,1,1], 'db1')
tx = np.linspace(0,1,150)
plt.plot(tx, cA, label='cA')
plt.plot(tx, cD, label='cD')
plt.plot(t, data[:,0,0], label='Orgin')
plt.legend(loc='upper right')
plt.show()

In [45]:
# related band power help function

def power_cal(data, band, trial, channel):
    y_data = data[:,trial, channel]

def power_cal(data, band, trial, channel):
    from scipy.integrate import simps
    y_data = data[:,trial,channel]
    low = band[0]
    high = band[1]
    #### 滤波
    b,a = signal.butter(5,[2*low/300,2*high/300],'bandpass')
    fil_sig = signal.filtfilt(b,a,data[:, trial, channel])
    b,a = signal.butter(5,[0.01,2*80/300],'bandpass')
    total_sig = signal.filtfilt(b,a,data[:, trial, channel])
    #### 计算功率
    sf = 300
#     win = 4*sf
    win = sf
    freqs, psd = signal.welch(fil_sig, sf, nperseg=win)
    idx_band = np.logical_and(freqs >= low, freqs <= high)
    band_power = simps(psd[idx_band], dx=freqs[1]-freqs[0])

    freqs, psd = signal.welch(total_sig, sf, nperseg=win)
    idx_total = np.logical_and(freqs >= 0, freqs <= 80)
    total_power = simps(psd[idx_total], dx=freqs[1]-freqs[0])
    
    result = band_power/total_power

    
    return result

In [46]:
# 计算各个频段的 related bandpower

# 数据结构：
# related_bandpower[sub-band x chanel x pic]

DELTA = [1,3]
THETA = [4,8]
ALPHA = [9,13]
BETA = [14,30]
sub_band = [DELTA, THETA, ALPHA, BETA]
pic = 2440 # 之后如果加入四个人可以修改
related_bandpower = np.zeros([len(sub_band), 19, pic])
for ch in range(19):
    for p in range(pic):
        for b in range(len(sub_band)):
            related_bandpower[b,ch,p] = power_cal(data, sub_band[b], p, ch)


In [47]:
# Standardization of each sub-band: zero mean and unit variance

scaled_related_bandpower = np.zeros([len(sub_band), 19, pic])
for b in range(len(sub_band)):
    for c in range(19):
        scaled_related_bandpower[b,:,:] = preprocessing.scale(related_bandpower[b,c,:])
#         transformer = preprocessing.RobustScaler().fit(related_bandpower[b,:,:])
#         scaled_related_bandpower[b,:,:] = transformer.transform(related_bandpower[b,:,:])

In [48]:
# PCA 降维: fix band, reduce chanel

# 数据结构：
# reduced [band x pic x chanel] (4, 2440, 5)

reduced = np.zeros([4,2440,5])

for band in range(4):
    temp = scaled_related_bandpower[band,:,:]
    t = np.zeros([2440,19])
    for i in range(2240):
        for j in range(19):
            t[i][j] = temp[j][i]

    randomized_pca = PCA(n_components=5, svd_solver='randomized')
    reduced[band,:,:] = randomized_pca.fit_transform(t)

reduced.shape

(4, 2440, 5)

### 观察不同人的反映程度

&emsp; 对于任何一张图片，4个人的四个band所展示的反映激烈程度可能会不一样，我们选择具有40轮实验的1号和3号人员，以及他们的PCA的第一主成分进行反映激烈程度的展示（下列结果是标准化之后的）：

&emsp; 结果可以发现，两个人都在theta和alpha波段有明显的能量，所以对这个实验影响最大的波段应该是theta和alpha，就是4-13Hz。

&emsp; 但是两者的能量分布有着比较明显的差异，在实际算法使用中还是选择比较小的波段范围，效果会更好（4-13Hz对于高维少数据量的样本来说还是太宽了），所以我们只选择一个人进行有针对性的训练。

&emsp; 并且可以看出，第三个人比第一个人具有更高的反映程度，实际训练结果也显示出第一个人的正确率较高，虽然只有0.1（展示在算法部分）

In [70]:
# std = True; person = 1; 四个类型波段能量占比

# reduced1 = reduced # run all above cell and choose one to comment 

power_mean1 = np.zeros([4,])
for band in range(4):
    power_mean1[band] = np.mean(reduced1[band,:,0])
power_mean1

array([-2.79557798e-16,  1.74723624e-16,  2.62085435e-16, -8.15376910e-17])

In [71]:
# std = True; person = 3; 四个类型波段能量占比

# reduced3 = reduced # run all above cell and choose one to comment 

power_mean3 = np.zeros([4,])
for band in range(4):
    power_mean3[band] = np.mean(reduced3[band,:,0])
power_mean3

array([-5.82412078e-18,  0.00000000e+00,  8.15376910e-17,  1.04834174e-16])

### Plot and filter: 

&emsp; 这部分主要展示了原始数据以及滤波之后的数据；并且一部分作为help function在接下来的代码中用到。



In [52]:
def filter_time(data,i):
    '''
    filter and output time domain y

    data: dataTrial1[:,n_trail,:] # (2100,19) one trail's brain wave
    i: data[:,i] # channel we want to see
    '''
    f = np.zeros((2100, 2), dtype=float)
    f[:,1] = data[:,i]
    f[:,0] = np.arange(2100)/300

    N = 2100 
    T = 7.0 / 2100.0
    x = f[:,0]
    y = f[:,1]

    yf = fft(y)
    xf = np.linspace(0.0, 1.0/(2.0*T), N//2)

    f_signal = rfft(y)
    W = fftfreq(y.size, d=x[1]-x[0])

    cut_f_signal = f_signal.copy()
    cut_f_signal[(W<1)] = 0 # filter all frequencies below 1

    cut_signal = irfft(cut_f_signal)
    
    return cut_signal


def filter_freq(data,i):
    '''
    filter and output freq domain y

    data: dataTrial1[:,n_trail,:] # (2100,19) one trail's brain wave
    i: data[:,i] # channel we want to see
    '''
    
    f = np.zeros((2100, 2), dtype=float)
    f[:,1] = data[:,i]
    f[:,0] = np.arange(2100)/300

    N = 2100 
    T = 7.0 / 2100.0 
    x = f[:,0]
    y = f[:,1]

    yf = fft(y)
    xf = np.linspace(0.0, 1.0/(2.0*T), N//2)

    f_signal = rfft(y)
    W = fftfreq(y.size, d=x[1]-x[0])

    cut_f_signal = f_signal.copy()
    cut_f_signal[(W<1)] = 0  # filter all frequencies below 1

    cut_signal = irfft(cut_f_signal)

    cut_signal_f = fft(cut_signal)

    return cut_signal_f

In [53]:
# plot_freq: plot_freq(track1_remember[1])
def plot_freq(n_trail,fl): 
    '''
    randomly choose from 40 trails
    plot frequency domain of 19 channel
    '''
    size = 5
    fig = plt.figure(figsize=(size,2*size))
    data = dataTrial1[:,n_trail,:] # (2100,19)
#     data = np.delete(data,7,1) # delete one row
    n_rows = 19
    n_samples = 2100

    ax = fig.add_subplot()
    ax.set_xlim(0, 7) # x's range: 0-7s
    ax.set_xticks(np.arange(7))
    dmin = 0
    dmax = 20
    dr = (dmax - dmin)
    y0 = dmin
    y1 = (n_rows - 1) * dr + dmax
    ax.set_ylim(y0, y1)

    N = 2100 
    T = 7.0 / 2100.0
    tf = np.linspace(0.0, 1.0/(2.0*T), N//2)
    
    segs = [] # list of lines, each line is an array of points
    for i in range(n_rows):
        if fl:
            yf = 2.0/N * np.abs(filter_freq(data,i)[:N//2])
            segs.append(np.column_stack((tf, yf)))  ### filter
        else:
            yf = 2.0/N * np.abs(data[:,i][:N//2])
            segs.append(np.column_stack((tf, yf)))
            

    offsets = np.zeros((n_rows, 2), dtype=float)
    offsets[:, 1] = np.arange(n_rows) * dr

    lines = LineCollection(segs, offsets=offsets, transOffset=None) # draw lines from segs
    ax.add_collection(lines)

    ax.set_yticks(np.arange(n_rows) * dr)
    ax.set_yticklabels(chanNameUsed)

    plt.grid(True)
    plt.show()

    
# plot_time: plot_freq(track1_not_remember[1])
def plot_time(n_trail,fl): 
    '''
    randomly choose from 40 trails
    plot time domain of 19 channel
    '''
    
    size = 10
    fig = plt.figure(figsize=(size,size))
    n_rows = 19 # delete one row
    n_samples = 2100
    data = dataTrial1[:,n_trail,:] # (2100,19)
#     data = np.delete(data,7,1)
    t = np.arange(2100) / 300

    ax = fig.add_subplot(1,1,1)
    ax.set_xlim(0, 7.01) # x's range: 0-7s
    ax.set_xticks(np.arange(7))
    dmin = data.min()
    dmax = data.max()
    dr = (dmax - dmin)
    y0 = dmin
    y1 = (n_rows - 1) * dr + dmax
    ax.set_ylim(y0, y1)

    segs = [] # list of lines, each line is an array of points
    for i in range(n_rows):
        if fl:
            segs.append(np.column_stack((t, filter_time(data,i))))  ### filter
        else:
            segs.append(np.column_stack((t, data[:,i])))

    offsets = np.zeros((n_rows, 2), dtype=float)
    offsets[:, 1] = np.arange(n_rows) * dr

    lines = LineCollection(segs, offsets=offsets, transOffset=None) # draw lines from segs
    ax.add_collection(lines)

    ax.set_yticks(np.arange(n_rows) * dr)
    ax.set_yticklabels(chanNameUsed)

    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [54]:
plot_time(1,False)
plot_time(1,True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

&emsp; 这部分是滤波的具体代码。之前通过查阅相关资料，我们首先锁定了theta和beta两个波段，所以函数cut的也是这两个；后面根据这个数据集算出对实验影响最大的是theta波段，所以最后启用的是theta波

&emsp; 首先，我们对原始信号进行傅立叶变换fft。

&emsp; 得到频谱图后，提取需要的相关频率(theta：4-8Hz，beta：14-30Hz）得到theta和beta波的频谱图。

&emsp; 重建信号，得到theta和beta波的时域图。

In [55]:
def cut_fs(data):
    '''
    data: (300,)
    result[0]: theta, [1]: beta, (300,)
    '''
    b,a = signal.butter(5,[8/300,16/300],'bandpass')
    theta = signal.filtfilt(b,a,data)
    b,a = signal.butter(5,[24/300,60/300],'bandpass')
    beta = signal.filtfilt(b,a,data)
    result = [theta,beta]
    return result


def cut_fss(dataTrial, trial, channel):
    """ 
    Input:  dataTrial channel 为处理的通道1-19
    Output: 提取看到图片后的数据3-6s。滤波后重建。
            result[0]: theta, [1]: beta
            并且画图
    """
    data = dataTrial[:,trial,:]  ##随便挑了一次trial,  900x19
    n_row = 19
    data_new = []
    t = np.linspace(0,3,900)
    for i in range(len(t)):
        data_new.append([t[i],data[i,channel]])
    data_new = np.array(data_new) ####### 900x2 ############
    fig1 = pylab.rcParams['figure.figsize'] = (15.0,2.0)
#     plt.plot(data_new[:,0],data_new[:,1])
#     plt.show()
    
    b,a = signal.butter(5,[8/300,16/300],'bandpass')
    theta = signal.filtfilt(b,a,data_new[:,1])
#     plt.plot(t,theta)
#     plt.title('theta')
#     plt.show()
    
    b,a = signal.butter(5,[24/300,60/300],'bandpass')
    beta = signal.filtfilt(b,a,data_new[:,1])
#     plt.plot(t,beta)
#     plt.title('beta')
#     plt.show()
    
    result = [theta,beta]
    
    N = 900
    Fs = 300
    ds = Fs/N
    yy = fft(theta)
    yf = abs(yy[:int(N)])/N
    yf = fftshift(yf) ##900
    freq = np.arange(-N/2,N/2)*ds ###900
#     plt.plot(freq, yf)
#     plt.title('theta')
#     plt.show()
    
    yy = fft(beta)
    yf = abs(yy[:int(N)])/N
    yf = fftshift(yf) ##900
    freq = np.arange(-N/2,N/2)*ds ###900
#     plt.plot(freq, yf)
#     plt.title('beta')
#     plt.show()
    
    return result

### 制作新的数据结构 (pair)

&emsp; 此外，从生物学角度出发，从19个通道中选取了部分可能相关的通道，两两排列组合构成了54个pair。

&emsp; 我们还尝试用不同的pair代替原有的19个channel作为新的输入，放到网络中训练。我们的pair也采用滤波后的theta波段作为输入。

In [76]:
# 数据结构：
# pair[pic, pair, data] 2440 x 54 x 300

# 与之前的label、data对应注释这格或者下一格

pair1 = np.zeros([2440,54,300])
for i in range(2440):
    for j in range(54):
        pair1[i][j] = cut_fs(data[:,i,pair54[j][0]])[0] - cut_fs(data[:,i,pair54[j][1]])[0]


In [74]:
pair3 = np.zeros([2440,54,300])
for i in range(2440):
    for j in range(54):
        pair3[i][j] = cut_fs(data[:,i,pair54[j][0]])[0] - cut_fs(data[:,i,pair54[j][1]])[0]


## 二、分类算法

&emsp; 在进行过数据预处理后，我们选用了10种算法对其进行分类，详细参数和内容如下列表所示。

In [120]:
classifiers = [
#     LogisticRegression(),
#     KNeighborsClassifier(2),
#     SVC(gamma=2, C=1, probability=True),
#     GaussianProcessClassifier(1.0 * RBF(1.0)),
#     DecisionTreeClassifier(max_depth=5),
#     RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
#     MLPClassifier(alpha=1, max_iter=100),
#     AdaBoostClassifier(),
    GaussianNB(),
#     QuadraticDiscriminantAnalysis(),
#     DummyClassifier(strategy='most_frequent', random_state=0)
]


&emsp; 在扩展数据集之后，KNN性价比最高（稳定而且运行速度快），所以最后选择KNN作为分析算法。


### 正确率计算：

&emsp; 测试两个人在扩展数据集之后的正确率。

&emsp; 根据预处理阶段的分析，我们预测第一个人的正确率应该更高，这与运行结果相符合。


In [101]:
# 2440个数据，KNN算法运行结果

models = pd.DataFrame(columns = classifiers)
test_score = np.zeros([54,])
test_probs = pd.DataFrame(columns = range(54))

for clf in classifiers:
    for p in range(54):
        X_train, X_test, y_train, y_test = train_test_split(pair3[:,p,:], label, test_size=0.25)
        clf.fit(X_train, y_train)
        test_score[p] = clf.score(X_test, y_test)
        test_probs[p] = clf.predict_proba(X_test)[:,0]
    models[clf] = test_score


In [100]:
# person1 18/19 pair 的正确率
# models1 = models

models1[18:20]

,KNeighborsClassifier(n_neighbors=2)
18,0.967213
19,0.962295


In [102]:
# person3 54 pair 的正确率
models3 = models

models3[18:20]

,KNeighborsClassifier(n_neighbors=2)
18,0.808197
19,0.837705


### 测试算法的置信程度：

&emsp; 与预测结果相符合——用40个数据求出的算法的置信度最高，而且波动也比较大；用2240个数据求出的算法置信度降低了一点；用PCA降维之后的2240个数据稳定在0.01左右。

In [157]:
# 40张的原始数据循环运行，输出pair18和19的正确率，观察稳定性

test_probs = pd.DataFrame(columns = range(54))

for clf in classifiers:
    for p in range(54):
        X_train, X_test, y_train, y_test = train_test_split(pair1[0:40,p,:], label[0:40], test_size=0.25)
        clf.fit(X_train, y_train)
        test_probs[p] = clf.predict_proba(X_test)[:,0]
test_probs.mean(axis=1).var()

0.008396766832840203

In [158]:
# 2440个数据循环运行，输出pair18和19的正确率，观察稳定性
test_probs = pd.DataFrame(columns = range(54))

for clf in classifiers:
    for p in range(54):
        X_train, X_test, y_train, y_test = train_test_split(pair1[:,p,:], label, test_size=0.25)
        clf.fit(X_train, y_train)
        test_probs[p] = clf.predict_proba(X_test)[:,0]
test_probs.mean(axis=1).var()

0.0037403122926294974

In [166]:
# 用PCA降维之后的 reduced [band x pic x chanel] (4, 2440, 5) 

test_probs = pd.DataFrame(columns = range(4))

for clf in classifiers:
    for b in range(4):
        X_train, X_test, y_train, y_test = train_test_split(reduced[b,:,:], label, test_size=0.25)
        clf.fit(X_train, y_train)
        test_probs[b] = clf.predict_proba(X_test)[:,0]
test_probs.mean(axis=1).var()

0.0016519589864122431

### 稳定性测试：

&emsp; 分别循环运行以2440和40个数据作为输入的KNN算法，观察score的变化，可以发现40张图片的算法波动很大而且误差较高，而扩增后的2440数据集则相对稳定得多，而且有更高的准确率。

In [103]:
# 2440个数据循环运行，输出pair18和19的正确率，观察稳定性

models = pd.DataFrame(columns = classifiers)
test_score = np.zeros([54,])

for i in range(10):
    for clf in classifiers:
        for p in range(54):
            X_train, X_test, y_train, y_test = train_test_split(pair1[:,p,:], label, test_size=0.25)
            clf.fit(X_train, y_train)
            test_score[p] = clf.score(X_test, y_test)
        models[clf] = test_score
    print(models[18:20])

    KNeighborsClassifier(n_neighbors=2)
18                             0.944262
19                             0.944262
    KNeighborsClassifier(n_neighbors=2)
18                             0.945902
19                             0.967213
    KNeighborsClassifier(n_neighbors=2)
18                             0.940984
19                             0.959016
    KNeighborsClassifier(n_neighbors=2)
18                             0.970492
19                             0.955738
    KNeighborsClassifier(n_neighbors=2)
18                             0.952459
19                             0.959016
    KNeighborsClassifier(n_neighbors=2)
18                             0.945902
19                             0.952459
    KNeighborsClassifier(n_neighbors=2)
18                             0.945902
19                             0.945902
    KNeighborsClassifier(n_neighbors=2)
18                             0.957377
19                             0.949180
    KNeighborsClassifier(n_neighbors=2)


In [104]:
# 40张的原始数据循环运行，输出pair18和19的正确率，观察稳定性

models = pd.DataFrame(columns = classifiers)
test_score = np.zeros([54,])
test_probs = pd.DataFrame(columns = range(54))

for i in range(10):
    for clf in classifiers:
        for p in range(54):
            X_train, X_test, y_train, y_test = train_test_split(pair1[0:40,p,:], label[0:40], test_size=0.25)
            clf.fit(X_train, y_train)
            test_score[p] = clf.score(X_test, y_test)
            test_probs[p] = clf.predict_proba(X_test)[:,0]
        models[clf] = test_score
    print(models[18:20])

    KNeighborsClassifier(n_neighbors=2)
18                                  0.2
19                                  0.4
    KNeighborsClassifier(n_neighbors=2)
18                                  0.4
19                                  0.4
    KNeighborsClassifier(n_neighbors=2)
18                                  0.2
19                                  0.5
    KNeighborsClassifier(n_neighbors=2)
18                                  0.3
19                                  0.4
    KNeighborsClassifier(n_neighbors=2)
18                                  0.4
19                                  0.5
    KNeighborsClassifier(n_neighbors=2)
18                                  0.6
19                                  0.6
    KNeighborsClassifier(n_neighbors=2)
18                                  0.4
19                                  0.4
    KNeighborsClassifier(n_neighbors=2)
18                                  0.6
19                                  0.3
    KNeighborsClassifier(n_neighbors=2)


## 三、结论与展望

&emsp; 在对原始数据进行预处理并分类后，我们发现，可以从原始的脑电信号中用适当的算法以较高的准确率判断出受试者是否记住了测试图像。其中，我们也发现预处理数据手法的不同对分类算法精度的影响是非常大的。因此，我们可以展望，在未来，只要以更加科学、精确的方式对脑电数据进行较好的预处理，我们便可以从较为粗糙的数据中得到非常实用的信息，这可以降低硬件的成本，为家庭医疗设施的普及做出较大的贡献。
